In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import keras
import pickle as pk
import matplotlib.pyplot as plt
import os
from skimage.transform import resize
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.layers import UpSampling2D, Input, Flatten, Lambda
from keras.models import Model, Sequential
from keras import regularizers
from keras import backend as K
from utils import *


/home/paperspace/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Define SiameseNet model

In [5]:
### Define model

resnet = ResNet50(weights='imagenet', include_top=False)
for layer in resnet.layers[:-12]:
    layer.trainable = False

# Def right and left inputs
left_img = Input(shape=(32, 32, 3), name='left_input')
right_img = Input(shape=(32, 32, 3), name='right_input')
left_inp = Lambda(lambda img: K.tf.image.resize_images(left_img, (224, 224)))(left_img)
right_inp = Lambda(lambda img: K.tf.image.resize_images(right_img, (224, 224)))(right_img)
left_out = resnet(left_inp)
right_out = resnet(right_inp)
diff = keras.layers.subtract([left_out, right_out])
diff2 = Lambda(lambda x: K.abs(x))(diff)
diff3 = Flatten()(diff2)

prediction = keras.layers.Dense(1,activation='sigmoid', name='dist',
                               kernel_regularizer=regularizers.l2(0.01))(diff3)
siamese_net = Model(inputs=[left_img,right_img],outputs=prediction)

# Compile model
adadelta = keras.optimizers.Adadelta(lr=0.001)
siamese_net.compile(loss='binary_crossentropy', optimizer=adadelta, metrics=['accuracy'])

# Print information about the model
siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_input (InputLayer)         (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
right_input (InputLayer)        (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224, 224, 3)  0           left_input[0][0]                 
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 224, 224, 3)  0           right_input[0][0]                
__________________________________________________________________________________________________
resnet50 (

### Feature extractor

In [ ]:
# Extract ResNet weight fine tuned
resnet_tuned = siamese_net.layers[4]
img = Input(shape=(32, 32, 3), name='input')
img2 = UpSampling2D(size =(7,7))(img)

# Def model
feat = resnet_tuned(img2)
extractor = Model(input=img, output=feat)
extractor.summary()

### Define custom distance for our k-NN

In [ ]:
# Extract distance layer from our model, and use it for the k-NN
dist_layer = siamese_net.get_layer('dist').get_weights()
def cust_dist(x, y):
    """Distance using the metric learned by our SiameseNet
    Args:
        x, y: (array (2048,)): the features to use
    return:
        dist: (float) distance value
    """
    temp = np.abs(x-y)
    return np.dot(dist_layer, temp)


### Siamese Networks training

In [ ]:
historyTrain = siamese_net.fit(dataset, dataset_labels, epochs=10,
                          batch_size=16, verbose=2, validation_split=0.2,
                          shuffle=True)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 246s - loss: 0.7573 - acc: 0.5017 - val_loss: 0.7946 - val_acc: 0.5078
Epoch 2/10


In [7]:
# Create toy dataset (to be improved)
data_path = 'cifar10/'
dataset = [[], []]
dataset_labels = []
cifar_batch_id = [1, 2, 3, 4 , 5]
nb_pairs_per_batch = 5000
for batch in cifar_batch_id:
    data, y = get_pairs(data_path, 0.5, batch, batch_size=nb_pairs_per_batch)
    dataset[0] += [data[0]]
    dataset[1] += [data[1]]
    dataset_labels += [y]
dataset[0] = np.concatenate(dataset[0])
dataset[1] = np.concatenate(dataset[1])
dataset_labels = np.concatenate(dataset_labels)

# Shuffle
idxes = np.arange(dataset_labels.shape[0])
np.random.shuffle(idxes)
dataset[0] = dataset[0][idxes]
dataset[1] = dataset[1][idxes]
dataset_labels = dataset_labels[idxes]

In [ ]:
# Create toy test dataset
data_path = 'cifar10/'
test_dataset = [None, None]
test_dataset_labels = []
nb_pairs_per_batch = 1000
data, y = get_pairs(data_path, 0.5, 'test', batch_size=nb_pairs_per_batch)
test_dataset[0] = data[0]
test_dataset[1] = data[1]
test_dataset_labels = y

# Shuffle
idxes = np.arange(test_dataset_labels.shape[0])
np.random.shuffle(idxes)
test_dataset[0] = test_dataset[0][idxes]
test_dataset[1] = test_dataset[1][idxes]
test_dataset_labels = test_dataset_labels[idxes]